In [31]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [32]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()


###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_video(video_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if video_path.startswith('http'):
        data_url = video_path
    else:
        mp4 = open(video_path, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(image_path, width=None, height=None):
    if isinstance(image_path, np.ndarray):
        img = image_path
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    response = requests.get(url)
    if response:
        imgmat = np.frombuffer(response.content, dtype=np.uint8)
        img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        if rgb:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if size:
           if isinstance(size, int):
               size = (size, size)
           img = cv2.resize(img, size)
        return img
    return None


In [33]:
import os
import csv
import json
import urllib
import requests
import random
import base64

from pprint import pprint
from random import sample
from urllib.request import urlretrieve
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL
from k12libs.utils.nb_easy import K12AI_DATASETS_ROOT
from IPython.display import display, HTML

In [34]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
def img2b64(path):
    with open(path, 'rb') as fr:
        b4data = base64.b64encode(fr.read()).decode()
    return b4data

## Base64 Input:

```json
{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.Base64DataLoader",
            "params": {
                "data_source": "b64str",
                "dataset": {
                    "class_name": "raceai.data.RawRaceDataset",
                    "params": {}
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 15,
               "iterations": 3,
               "edge_thickness": 30
           }
       }
    }
}
```


## List Path Input:

```json
{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [{"image_id": 1, "image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/dining_plate/blue.png"}],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageRaw",
                    "params": {}
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 15,
               "iterations": 3,
               "edge_thickness": 30
           }
       }
    }
}
```

In [35]:
reqdata = '''{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [{"image_id": 1, "image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/dining_plate/blue.png"}],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageRaw",
                    "params": {}
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 15,
               "iterations": 3,
               "edge_thickness": 30
           }
       }
    }
}'''
# }''' % (img2b64('/data/nb_data/cv/plate_color/blue.png'))

print(reqdata)

{
    "task": "plate_color.inference",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [{"image_id": 1, "image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/dining_plate/blue.png"}],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageRaw",
                    "params": {}
                 }
             }
        },
        "model": {
           "class_name": "raceai.models.xfeatures.PlateColorFeatures",  
           "params": {
               "ellsize": 15,
               "iterations": 3,
               "edge_thickness": 30
           }
       }
    }
}


## Output:

```

colors_names = ['black', 'gray', 'white', 'red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple']


{
    "errno": 0,
    "result": [
        {
            "argmax": 8,                // 颜色数组的的下标, 8:blue
            "image_id": 1,              // 传递过来的图标id
            "image_path": "/tmp/tmpnfjfhwdh/blue.png",  // 本地调试使用
            "probs": [                                  // 概率数组, 概率最大的下标就是最终答案.
                0.0023584905660377358,
                0.0,
                0.030660377358490566,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.9669811320754716,
                0.0
            ]
        }
    ]
}
```

In [37]:
cfg = eval(reqdata)
resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
print(json.dumps(resdata, indent=4))

{
    "errno": 0,
    "result": [
        {
            "argmax": 8,
            "image_id": 1,
            "image_path": "/tmp/tmpbs4_xivk/blue.png",
            "probs": [
                0.0023584905660377358,
                0.0,
                0.030660377358490566,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.9669811320754716,
                0.0
            ]
        }
    ]
}
